In [2]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai

In [3]:
import os
from getpass import getpass

In [4]:
os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API Key: ")

Enter your Mistral API Key: ··········


In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/2022-GROUP-II_20SERVICES-NOTIFN_20FINAL2022122923474120230118102628 (1).pdf")  # upload PDF to Colab
documents = loader.load()

print(f"Total pages loaded: {len(documents)}")

Total pages loaded: 46


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks = text_splitter.split_documents(documents)
print(f"Total chunks created: {len(chunks)}")

Total chunks created: 300


In [7]:
from langchain_mistralai.embeddings import MistralAIEmbeddings

embedding_model = MistralAIEmbeddings(
    model="mistral-embed"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

In [8]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

print("Vector store created successfully")

Vector store created successfully


In [9]:
query = "What are the Physical Requirements for Prohibition and Excise Sub Inspector? "

results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content[:400])


--- Chunk 1 ---
Caste Development  Officer 
in Scheduled Caste 
Development Department 
Must possess a Bachelor’s Degree of a University 
in India or an equivalent qualification. 
 
For Pc. No. 08 :- Physical Requirements for Prohibition and Excise Sub Inspector  
 
MEN:   
 i)  Height: Must not be less than 167.6 Centimeters. 
 ii) Chest: Must not be less than 8 6.3 Centimeters round the chest on full inspiratio

--- Chunk 2 ---
MULTI 
ZONE-ll 12 7 3 2 3 2 2 2 1 - 2+1CF 1 1 1 5 3 3 2 1 - 1 1 - - - - - 35 21 56 
TOTAL 26 15 7 5 6 4 5 4 2 - 6 4 3 2 11 7 8 4 2 - 2 2 - 1 - -  -  79 47 126 
 
 
 
BREAK UP OF VACANCIES FOR THE POST OF PROHIBITION AND EXCISE SUB INSPECTOR IN PROHIBITION & EXCISE DEPARTMENT 
 
POST CODE-08 
 
CONTIGUOUS ZONE 
OC EWS BC-A BC-B BC-C BC-D BC-E SC ST SPORTS TOTAL GRAND 
TOTAL G W G W G W G W G W G W 

--- Chunk 3 ---
Mandal Panchayat Officer 
[Extension Officer] in 
Panchayat Raj and Rural 
Development Department 
Must have possessed a Bachelor’s Degree from 
an

In [10]:
from langchain_mistralai.chat_models import ChatMistralAI


llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0
)


In [12]:
def generate_answer(query, retrieved_docs):
    context = "\n\n".join(
        [f"[Chunk {i+1}]: {doc.page_content}"
         for i, doc in enumerate(retrieved_docs)]
    )

    prompt = f"""
You are a factual assistant.

Answer the question using ONLY the context below.
If the answer is not present, say:
"Answer not found in the provided document."

CONTEXT:
{context}

QUESTION:
{query}

Include citations like [Chunk X].
"""

    response = llm.invoke(prompt)
    return response.content


Threshold value 0.9

In [23]:
SIMILARITY_THRESHOLD = 0.9

def retrieve_with_threshold(query, k=5):
    docs_with_scores = vectorstore.similarity_search_with_score(query, k=k)

    filtered_docs = []
    for doc, score in docs_with_scores:
        similarity = 1 / (1 + score)  # FAISS distance → similarity
        if similarity >= SIMILARITY_THRESHOLD:
            filtered_docs.append(doc)

    return filtered_docs


In [24]:
def rag_pipeline(query):
    retrieved_docs = retrieve_with_threshold(query)

    if len(retrieved_docs) == 0:
        return "Naku telsindhi adugu broooooooo!"

    return generate_answer(query, retrieved_docs)


In [25]:
query = "What is the qualification ?"
answer = rag_pipeline(query)
print(answer)

Naku telsindhi adugu broooooooo!


In [26]:
query = "What is Relaxation of age permissible for Ex-Service men  ?"
answer = rag_pipeline(query)
print(answer)

Naku telsindhi adugu broooooooo!


Threshold value 0

In [27]:
SIMILARITY_THRESHOLD = 0

def retrieve_with_threshold(query, k=5):
    docs_with_scores = vectorstore.similarity_search_with_score(query, k=k)

    filtered_docs = []
    for doc, score in docs_with_scores:
        similarity = 1 / (1 + score)  # FAISS distance → similarity
        if similarity >= SIMILARITY_THRESHOLD:
            filtered_docs.append(doc)

    return filtered_docs


In [28]:
def rag_pipeline(query):
    retrieved_docs = retrieve_with_threshold(query)

    if len(retrieved_docs) == 0:
        return "Naku telsindhi adugu broooooooo!"

    return generate_answer(query, retrieved_docs)

In [29]:
query = "What is the qualification ?"
answer = rag_pipeline(query)
print(answer)

The qualification for the post of **Municipal Commissioner Gr.III in Municipal Administration Department** is specified in **[Chunk 5]** as:

**"Applicants must possess the requisite qualifications as detailed in the relevant Service Rules, indented by the Department."**

However, the exact educational qualification for this specific post is **not explicitly mentioned** in the provided context.

**Answer not found in the provided document.**


In [30]:
query = "What is Relaxation of age permissible for Ex-Service men  ?"
answer = rag_pipeline(query)
print(answer)

The relaxation of age permissible for Ex-Service men is **3 years plus the length of service rendered in the armed forces**, provided they were not released by way of dismissal or discharge on account of misconduct or inefficiency.

Citations:
[Chunk 1], [Chunk 2], [Chunk 3]


Threshold value -1

In [31]:
SIMILARITY_THRESHOLD = 0

def retrieve_with_threshold(query, k=5):
    docs_with_scores = vectorstore.similarity_search_with_score(query, k=k)

    filtered_docs = []
    for doc, score in docs_with_scores:
        similarity = 1 / (1 + score)  # FAISS distance → similarity
        if similarity >= SIMILARITY_THRESHOLD:
            filtered_docs.append(doc)

    return filtered_docs


In [32]:
def rag_pipeline(query):
    retrieved_docs = retrieve_with_threshold(query)

    if len(retrieved_docs) == 0:
        return "Naku telsindhi adugu broooooooo!"

    return generate_answer(query, retrieved_docs)

In [33]:
query = "What is Relaxation of age permissible for Ex-Service men  ?"
answer = rag_pipeline(query)
print(answer)

The relaxation of age permissible for Ex-Service men is **3 years plus the length of service rendered in the armed forces**, provided they were not released by way of dismissal or discharge due to misconduct or inefficiency.

Citations:
[Chunk 1], [Chunk 2], [Chunk 3]
